In [1]:
# Importando bibliotecas
import numpy as np
import pandas as pd
import sys

In [2]:
# Path referente aa localizacao da pasta modules
#sys.path.append('/home/rodbijani/Documents/UFF/graduacao/IC/BrunoLima/IC-UFF/modules') # pc do Bijani
sys.path.append('C:/Users/bruno/Documents/UFF/Projeto_Bijani/GitHub/IC-UFF/modules') # pc do Bruno

In [3]:
# Importando bibliotecas extras
import prism

In [4]:
# Lendo as coordenadas dos pontos de observacao e os valores de anomalia de campo gravitacional observados
df = pd.read_csv('log_file.txt', sep='\s+', header=19) # \s+ para multiplos espaços

In [5]:
# Visualizando informacoes lidas
df

,X(m),Y(m),Z(m),gz_obs(mGal)
0,0.000000,0.0,0.0,-0.795409
1,504.201681,0.0,0.0,-0.903162
2,1008.403361,0.0,0.0,-1.190305
3,1512.605042,0.0,0.0,-2.637764
4,2016.806723,0.0,0.0,-2.967793
...,...,...,...,...
115,57983.193277,0.0,0.0,-2.819352
116,58487.394958,0.0,0.0,-3.034851
117,58991.596639,0.0,0.0,-2.716049
118,59495.798319,0.0,0.0,-0.942139


In [6]:
# Organizando as informacoes extras do arquivo lido ja pensando na funcao onde serao utilizadas
deltax = 504.20168067226894 # devidamente descrito no arquivo lido
deltay = 1000.0             # devidamente descrito no arquivo lido

In [7]:
# Definindo dados constantes para o metodo iterativo

n_obs = len(df['gz_obs(mGal)']) # numero de observacoes

gama = 6.67                  # constante universal da gravitacao no S.I. utilizada na referencia Rafael-Cristo.pdf
#gama_SI = 6.673e-11          # constante universal da gravitacao no S.I.
#si2mGal = 100000.0           # fator para converter a constante universal da gravitacao para o sistema cgs
#gama_cgs = gama_SI * si2mGal # constante universal da gravitacao no sistema cgs

delta_rho = np.array([-0.24])   # contraste de densidade em g/cm³
#delta_rho_SI = delta_rho * 1000 # contraste de densidade em kg/m³

#gz_obs_SI = df['gz_obs(mGal)'] / 100000 # valores de anomalia observados em m/s²

In [9]:
# Metodo iterativo proposto por Bott (1960) para calcular a profundidade a partir da anomalia observada

# Criando vetor que armazena os valores de profundidade
z_bottom = np.zeros(n_obs) 

# Criando vetor que armazena os valores de campo gravitacional
gz_calc = np.zeros(n_obs)
 
# Definindo contador do numero de iteracoes
# Contador permite um criterio de parada de acordo com o numero de iteracoes
interacoes = 0

# Definindo variavel que recebera o valor do erro
error = 0

# Define o tipo de erro adotado
type_error = 1 

start = pd.Timestamp.now() # Iniciando a contagem do tempo para rodar o metodo iterativo

####################################################################################################
############################################### Inicio #############################################
####################################################################################################

# Criterio de parada
#while contador != n_obs: # Definido pela quantidade de valores calculados de anomalia que são satisfatorios
while interacoes < 3000: # Definido pelo numero de iteracoes   
#while error < 0.00001  # Definido pelo erro
    
    # Adicionando mais uma contagem de iteracao a medida que elas ocorrem
    interacoes += 1
    
    #print('Interações: ', interacoes, '. Contador: ', contador, '. Objetivo: ', n_obs)
    print('Interacoes: ', interacoes)
    
    # Contador do numero de valores calculados de anomalia que são satisfatorios
    # Resentando o contador para verificar a cada iteracao se todos os valores estao satisfatorios
    # contador = 0
    ######################################## Primeira etapa ########################################
    ################################### Ajuste da profundidade #####################################
    
    if interacoes == 1:
        # Aproximacao inicial para as profundidades
        for j in range (n_obs):
            z_bottom[j] = (df['gz_obs(mGal)'][j] / (2 * np.pi * gama * delta_rho[0])) / 1000
    else:
        # Atualizacao da profundidade
        for j in range (n_obs): 
            z_bottom[j] = z_bottom[j] + (((df['gz_obs(mGal)'][j] - gz_calc[j]) / (2 * np.pi * gama * delta_rho[0]))) / 1000 
              
    print('Z_bottom(1): ', z_bottom[0], '. Z_bottom(60): ', z_bottom[60])
    ######################################### Segunda etapa ########################################
    ################################# Calculo da anomalia modelada #################################    
     
    # Os valores de campo foram calculados através da modelagem 2-D proposta por Last e Kubik (1983)       
    
    # Calculando a componente vertical no eixo z da anomalia do campo gravitacional    
    gz_calc = prism.prism3D_gz(df['X(m)'], df['Y(m)'], df['Z(m)'], 
                               df['X(m)'], df['Y(m)'], 
                               df['Z(m)'], z_bottom, deltax, deltay, delta_rho)
    
    #gz_calc_SI = gz_calc / 100
    print('Gz Calc: ', gz_calc[0], '. Gz Obs: ', df['gz_obs(mGal)'][0], '\n')
    
    ######################################## Terceira etapa ########################################
    ###################################### Avaliacao do erro ####################################### 
    
    if type_error == 1: 
        error = 0
        std = df['gz_obs(mGal)'].std()
        #std = 0.1
        for i in range (n_obs):
            error += (1/n_obs)*((gz_calc[i] - df['gz_obs(mGal)'][i])**2)/(std ** 2)
    
    elif type_error == 2: # Erro quadratico medio definido na referencia de Rafael Cristo
        error = 0
        for i in range (n_obs):
            error += np.sqrt(((df['gz_obs(mGal)'][i] - gz_calc[i])**2)/(df['gz_obs(mGal)'][i]**2))
    
    print('Erro: ', error)
####################################################################################################
################################################ Fim ###############################################
####################################################################################################

end = pd.Timestamp.now() - start # Tempo para rodar o metodo iterativo

Interacoes:  1
Z_bottom(1):  7.908128801772349e-05 . Z_bottom(60):  0.0007685495195506719
Gz Calc:  -7.957663455231813e-07 . Gz Obs:  -0.7954093250228322 

Erro:  6.91648609804268
Interacoes:  2
Z_bottom(1):  0.00015816249691866322 . Z_bottom(60):  0.0015370982702065288
Gz Calc:  -1.591532854959485e-06 . Gz Obs:  -0.7954093250228322 

Erro:  6.916472258871733
Interacoes:  3
Z_bottom(1):  0.0002372436267028029 . Z_bottom(60):  0.002305646251969151
Gz Calc:  -2.387299314678064e-06 . Gz Obs:  -0.7954093250228322 

Erro:  6.916458419740556
Interacoes:  4
Z_bottom(1):  0.0003163246773701475 . Z_bottom(60):  0.003074193464839714
Gz Calc:  -3.1830627773500737e-06 . Gz Obs:  -0.7954093250228322 

Erro:  6.916444580648476
Interacoes:  5
Z_bottom(1):  0.0003954056489209949 . Z_bottom(60):  0.0038427399088207205
Gz Calc:  -3.978826019399681e-06 . Gz Obs:  -0.7954093250228322 

Erro:  6.916430741594646
Interacoes:  6
Z_bottom(1):  0.0004744865413553671 . Z_bottom(60):  0.00461128558391291
Gz Calc:

Gz Calc:  -3.660438671321921e-05 . Gz Obs:  -0.7954093250228322 

Erro:  6.9158633738815425
Interacoes:  47
Z_bottom(1):  0.0037167350129080788 . Z_bottom(60):  0.03612099626874013
Gz Calc:  -3.740011288759798e-05 . Gz Obs:  -0.7954093250228322 

Erro:  6.91584953646083
Interacoes:  48
Z_bottom(1):  0.0037958125825269776 . Z_bottom(60):  0.036889509651910776
Gz Calc:  -3.819584091862299e-05 . Gz Obs:  -0.7954093250228322 

Erro:  6.915835699079655
Interacoes:  49
Z_bottom(1):  0.003874890073032902 . Z_bottom(60):  0.03765802226625941
Gz Calc:  -3.899156675585345e-05 . Gz Obs:  -0.7954093250228322 

Erro:  6.915821861737357
Interacoes:  50
Z_bottom(1):  0.0039539674844260705 . Z_bottom(60):  0.03842653411178714
Gz Calc:  -3.978729114971621e-05 . Gz Obs:  -0.7954093250228322 

Erro:  6.915808024433812
Interacoes:  51
Z_bottom(1):  0.0040330448167066255 . Z_bottom(60):  0.03919504518849648
Gz Calc:  -4.058301291320063e-05 . Gz Obs:  -0.7954093250228322 

Erro:  6.915794187169045
Interacoe

Gz Calc:  -7.320693156420216e-05 . Gz Obs:  -0.7954093250228322 

Erro:  6.915226892789287
Interacoes:  93
Z_bottom(1):  0.0073542213349405946 . Z_bottom(60):  0.07147181618741263
Gz Calc:  -7.400261727301441e-05 . Gz Obs:  -0.7954093250228322 

Erro:  6.915213057158506
Interacoes:  94
Z_bottom(1):  0.007433295265460604 . Z_bottom(60):  0.07224029420637822
Gz Calc:  -7.479830479652355e-05 . Gz Obs:  -0.7954093250228322 

Erro:  6.91519922156422
Interacoes:  95
Z_bottom(1):  0.0075123691168716665 . Z_bottom(60):  0.07300877145659312
Gz Calc:  -7.559399237130403e-05 . Gz Obs:  -0.7954093250228322 

Erro:  6.915185386009018
Interacoes:  96
Z_bottom(1):  0.007591442889173777 . Z_bottom(60):  0.07377724793805918
Gz Calc:  -7.638967831472178e-05 . Gz Obs:  -0.7954093250228322 

Erro:  6.915171550494429
Interacoes:  97
Z_bottom(1):  0.007670516582367097 . Z_bottom(60):  0.07454572365077748
Gz Calc:  -7.718536084159962e-05 . Gz Obs:  -0.7954093250228322 

Erro:  6.91515771501824
Interacoes:  9

Gz Calc:  -0.00010980764221336253 . Gz Obs:  -0.7954093250228322 

Erro:  6.914590493961488
Interacoes:  139
Z_bottom(1):  0.010991540262526646 . Z_bottom(60):  0.10682100942644841
Gz Calc:  -0.00011060329116990074 . Gz Obs:  -0.7954093250228322 

Erro:  6.91457666011807
Interacoes:  140
Z_bottom(1):  0.011070610554129756 . Z_bottom(60):  0.10758945208448849
Gz Calc:  -0.00011139893385579091 . Gz Obs:  -0.7954093250228322 

Erro:  6.914562826312119
Interacoes:  141
Z_bottom(1):  0.011149680766628377 . Z_bottom(60):  0.10835789397384918
Gz Calc:  -0.00011219458086090837 . Gz Obs:  -0.7954093250228322 

Erro:  6.914548992546134
Interacoes:  142
Z_bottom(1):  0.01122875090002208 . Z_bottom(60):  0.10912633509453241
Gz Calc:  -0.00011299022666037111 . Gz Obs:  -0.7954093250228322 

Erro:  6.914535158820003
Interacoes:  143
Z_bottom(1):  0.011307820954310984 . Z_bottom(60):  0.10989477544653968
Gz Calc:  -0.00011378587140022481 . Gz Obs:  -0.7954093250228322 

Erro:  6.914521325131498
Inter

KeyboardInterrupt: 